Author: Behzad Barati

Abstract:

*   Data is about amont of rain in Lake_Bilancino area and flow rate of this lake in Italy.

My Question:
*   what is the relationship between amont of rains and flow rates 

___
Hint:

As PySpark has some issues with windows machines, I prefer to use cloud services(here google colab). I uploaded  dataset in my google drive. If you use regular machines, You can ignore mounting step.

# Import needed libraries

In [2]:
# Install pandas_profiling library
# pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive              # Mount google drive to colab notebook


#pip install pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as function


In [3]:
# Mount google drive to colab notebook
# Our dataset will be read as recipe_table.

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
cd gdrive/MyDrive/Projects/Assign4-Prog2

/content/gdrive/MyDrive/Projects/Assign4-Prog2


# Pyspark practice

In [5]:
spark = SparkSession.builder.appName("myFirstCSVLoad").getOrCreate()

In [6]:
df = spark.read.csv(path= './Lake_Bilancino.csv', sep= ',', header= True)

In [7]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Rainfall_S_Piero: string (nullable = true)
 |-- Rainfall_Mangona: string (nullable = true)
 |-- Rainfall_S_Agata: string (nullable = true)
 |-- Rainfall_Cavallina: string (nullable = true)
 |-- Rainfall_Le_Croci: string (nullable = true)
 |-- Temperature_Le_Croci: string (nullable = true)
 |-- Lake_Level: string (nullable = true)
 |-- Flow_Rate: string (nullable = true)



In [8]:
df.show()

+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+
|      Date|Rainfall_S_Piero|Rainfall_Mangona|Rainfall_S_Agata|Rainfall_Cavallina|Rainfall_Le_Croci|Temperature_Le_Croci|Lake_Level|Flow_Rate|
+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+
|03/06/2002|            null|            null|            null|              null|             null|                null|    249.43|     0.31|
|04/06/2002|            null|            null|            null|              null|             null|                null|    249.43|     0.31|
|05/06/2002|            null|            null|            null|              null|             null|                null|    249.43|     0.31|
|06/06/2002|            null|            null|            null|              null|             null|                null|    249.43|     0.31|

In [9]:
df.count()

6603

In [10]:
df.columns

['Date',
 'Rainfall_S_Piero',
 'Rainfall_Mangona',
 'Rainfall_S_Agata',
 'Rainfall_Cavallina',
 'Rainfall_Le_Croci',
 'Temperature_Le_Croci',
 'Lake_Level',
 'Flow_Rate']

In [11]:
# drop rows which have NA values

df = df.na.drop()
df.show()

+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+
|      Date|Rainfall_S_Piero|Rainfall_Mangona|Rainfall_S_Agata|Rainfall_Cavallina|Rainfall_Le_Croci|Temperature_Le_Croci|Lake_Level|Flow_Rate|
+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+
|02/01/2004|               0|               0|               0|                 0|                0|                 6.5|    251.21|      0.5|
|03/01/2004|               0|               0|               0|                 0|                0|                4.45|    251.28|      0.5|
|04/01/2004|               0|               0|               0|                 0|                0|                   2|    251.35|      0.5|
|05/01/2004|             0.2|               0|               0|                 0|                0|                 0.9|    251.37|      0.5|

In [12]:
# list of columns related to amont of rain

Rain_columns = ['Rainfall_S_Piero', 'Rainfall_Mangona', 'Rainfall_S_Agata', 'Rainfall_Cavallina', 'Rainfall_Le_Croci',]

In [13]:
# Sum multiple colomns to create new Sum of rains

df = df.withColumn('sum_rain', sum(df[col] for col in Rain_columns))
df.show()

+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+------------------+
|      Date|Rainfall_S_Piero|Rainfall_Mangona|Rainfall_S_Agata|Rainfall_Cavallina|Rainfall_Le_Croci|Temperature_Le_Croci|Lake_Level|Flow_Rate|          sum_rain|
+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+------------------+
|02/01/2004|               0|               0|               0|                 0|                0|                 6.5|    251.21|      0.5|               0.0|
|03/01/2004|               0|               0|               0|                 0|                0|                4.45|    251.28|      0.5|               0.0|
|04/01/2004|               0|               0|               0|                 0|                0|                   2|    251.35|      0.5|               0.0|
|05/01/2004|             0.2

In [14]:
df.withColumn('Rainy_day', function.when(function.col('sum_rain') > 270, "Yes").otherwise("No")).show()

+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+------------------+---------+
|      Date|Rainfall_S_Piero|Rainfall_Mangona|Rainfall_S_Agata|Rainfall_Cavallina|Rainfall_Le_Croci|Temperature_Le_Croci|Lake_Level|Flow_Rate|          sum_rain|Rainy_day|
+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+------------------+---------+
|02/01/2004|               0|               0|               0|                 0|                0|                 6.5|    251.21|      0.5|               0.0|       No|
|03/01/2004|               0|               0|               0|                 0|                0|                4.45|    251.28|      0.5|               0.0|       No|
|04/01/2004|               0|               0|               0|                 0|                0|                   2|    251.35|      0.

In [15]:
# Filter data based on if we have very rainy day. but i do not want to rewrite this df.

df.filter('sum_rain > 350').show()

+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+--------+
|      Date|Rainfall_S_Piero|Rainfall_Mangona|Rainfall_S_Agata|Rainfall_Cavallina|Rainfall_Le_Croci|Temperature_Le_Croci|Lake_Level|Flow_Rate|sum_rain|
+----------+----------------+----------------+----------------+------------------+-----------------+--------------------+----------+---------+--------+
|22/12/2009|            71.8|            90.6|            77.8|                77|             61.2|                 4.7|     246.4|      0.6|   378.4|
|20/09/2014|            52.4|            56.4|           102.4|             113.6|             51.6|                21.1|    248.96|        1|   376.4|
|15/09/2016|            80.6|           102.8|            83.8|              91.8|             66.6|               19.75|    247.77|        3|   425.6|
|09/09/2017|            52.8|            86.6|            69.4|              86.8|      